<a href="https://colab.research.google.com/github/cing3000/Reinforcement-Learning-An-Introduction/blob/master/Reinforcement_Learning_Notebook_Chapter_8_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 8: Planning and Learning with Tabular Methods

Methods that require a model of the environment, such as dynamic programming and heuristic search, are called _model-based_ reinforcement learning methods.

Methods that can be used without a model, such Monte Carlo and temporal-difference methods, are called _model-free_ reinforcement learning methods.

Model-based methods rely on _planning_ as their primary component, while model-free methods primarily rely on _learning_.

## 8.1 Models and Planning

Distribution models: produce a description of all possibilities and their probabilities.
Sample models: product just one of the possibilities, sampled according to the probabilities.

We use the term _planning_ to refer to any computational process that takes a model as input and produces or imporoves a policy for interacting with the modeled environment.

State-space planning: is viewed primarily as a search through the state space for an optimal policy or an optimal path to a goal.

Plan-space planning: **?**

This common structure can be diagrammed as follows:

<img src="https://cdn-images-1.medium.com/max/1200/1*_mm9H3rRvaTseXEim47saw.png" width="800"/>

<font color='red'>The heart of both learning and planning methods is the estimation of value functions by backing-up update operations. The difference is that whereas planning uses simulated experience generated by a model, learning methods use real experience generated by the environment.</font>

** Random-sample one-step tabular Q-planning **

---
Do forever:
1. Select a state, $S\in\mathcal{S}$, and an action, $A \in\mathcal{A}(s)$, at random
1. Send $S$, $A$ to a sample model, and obtain: a sample next reward, $R$, and a sample next state, $S'$
1. Apply one-step tabular Q-learning to $S$, $A$, $R$, $S'$:
>$Q(S,A)\leftarrow Q(S,A)+\alpha\big[R+\gamma \max_aQ(S',a)-Q(S,A)\big]$
---


## 8.2 Dyna: Integrating Plannning, Acting and Learning

**Dyna-Q**: a simple architecture integrating the major functions needed in an on-line planning agent.

<img src="https://cdn-images-1.medium.com/max/800/1*XU8ikxAFcn942MueCXK1aw.png" width="600" />

Dyna-Q includes all of the processes shown in Figure 8.1, all occurring continuously. The planning method is the random-sample one-step tabular Q-planning method. The direct RL method is one-step tabular Q-learning. The model-learning method is also table-based and assumes the environment is deterministic. After each transition $S_t,A_t \rightarrow R_{t+1},S_{t+1}$, the model records in its table entry for $S_t, A_t$ the prediction that $R_{t+1}, S_{t+1}$ will deterministically follow.